# Demo notebook: Clojupyter in Jupyter Lab

This notebook demonstrates some of the more advanced features of Clojupyter.  Please note that this notebook is intended to be used with **Jupyter Lab**, see Jupyter Notebook demo for details.


## Displaying HTML

To display HTML, you'll need to require a clojupyter helper function to change the cell output:

In [ ]:
(require '[clojupyter.display :as display])
;; print various data: Date & Time, Clojure version, Clojupyter version
(println (str "Date:\t\t\t"(java.util.Date.)))
(println (apply format "Clojure version:\tv%d.%d" ((juxt :major :minor) *clojure-version*)))
(println (str "Clojupyter version:\t" (:formatted-version clojupyter/*version*)));

We can use [Hiccup](https://github.com/weavejester/hiccup) to render HTML:

In [ ]:
;; displaying html
(display/hiccup-html 
    [:ul 
     [:li "a " [:i "emphatic"] " idea"]
     [:li "a " [:b "bold"] " idea"]
     [:li "an " [:span {:style "text-decoration: underline;"} "important"] " idea"]])

...which also works for SVGs:

In [ ]:
(display/hiccup-html
    [:svg {:height 100 :width 100 :xmlns "http://www.w3.org/2000/svg"}
            [:circle {:cx 50 :cy 40 :r 40 :fill "red"}]])

## Displaying bitmaps

Clojupyter can display bitmaps directly:

In [ ]:
(->> clojupyter/*logo* type (str "Logo is of type: ") println)
clojupyter/*logo*

## Adding External Clojure Dependencies

You can fetch external Clojure dependencies using the `clojupyter.misc.helper` namespace.

In [ ]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[org.clojure/data.json "0.2.6"])
:ok

## Charting using Clojure visualization library Oz

[Oz](https://github.com/metasoarous/oz) is a data visualization and scientific document processing library for Clojure built around [Vega Lite](https://vega.github.io/vega-lite/) & [Vega](https://vega.github.io/).  Jupyter Lab has direct support for rendering Vega and Vega Lite charts using the renderer for mime-type (this cell typically takes a little time to evaluate since it loads a library, either by retrieving it from [Clojars](www.clojars.org), or from the file system):

In [ ]:
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(require '[oz.notebook.clojupyter :as oz])
:ok

For this demo we define some simple functions generating some data compatible with the high-level  charting library:

In [ ]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :encoding {:x {:field :Index}
                                 :y {:field :Value}}})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

With a generator for (slightly) random data...

In [ ]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

...we can create a graph using Vega Lite:

In [ ]:
(let [G (->> [(range) (take 150 (raw-data))] graph-it)]
  (display/render-mime :application/vnd.vegalite.v2+json G))

**Note**: The above cell is *not* supposed to render correctly in Jupyter Notebook, <font color=red>**only Jupyter Lab**</font>.

# Incanter Plotting Examle in Clojupyter

If a cell returns a Java [BufferedImage](https://docs.oracle.com/javase/7/docs/api/java/awt/image/BufferedImage.html), then Clojupyter will render the image. This makes it easy to use Java charting libraries, like [Incanter](https://github.com/incanter/incanter), where the chart can be converted into an image easily. First let's bring in some helper namespaces and bring in Incanter:

In [ ]:
(helper/add-dependencies '[incanter "1.5.7"])
;(helper/add-dependencies '[org.clojure/math.numeric-tower "0.0.4"])
;(require '[clojure.core.reducers :as r])
(use '(incanter core stats charts io))
:ok

Since Incanter simply wraps the Java charting library [JFreeChart](https://github.com/incanter/incanter), we can call 
`(.createBufferedImage chart width height)` on any Incanter chart to get an imafge we can render as cell output:

In [ ]:
(-> (sample-normal 10000)
    histogram
    (.createBufferedImage 600 400))

Here's an example of a scatter plot:

In [ ]:
(-> (scatter-plot (sample-normal 1000) 
                  (sample-normal 1000)
                  :x-label "x" :y-label "y")
    (.createBufferedImage 600 400)) 